## Read in Data

In [ ]:
import joblib
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

tr_features = pd.read_csv('../../../train_features.csv')
tr_labels = pd.read_csv('../../../train_labels.csv')['Survived']

In [ ]:
KNeighborsClassifier()

## Hyperparameter tuning

Following the ideas in https://medium.com/@mohtedibf/in-depth-parameter-tuning-for-knn-4c0de485baf6

In [ ]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [ ]:
kn = KNeighborsClassifier()
parameters = {
    'n_neighbors': list(range(1, 30)),
    'p': [1, 2, 3, 4, 5] # Power Parameter for Minkowski metric
}

cv = GridSearchCV(kn, parameters, cv=5)
cv.fit(tr_features, tr_labels.array)

In [ ]:
print_results(cv)

In [ ]:
cv.best_estimator_

## Write out pickled model

In [ ]:
joblib.dump(cv.best_estimator_, '../../../KNN_model.pkl')

## Read in More Data

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from time import time

val_features = pd.read_csv('../../../val_features.csv')
val_labels = pd.read_csv('../../../val_labels.csv')['Survived']

te_features = pd.read_csv('../../../test_features.csv')
te_labels = pd.read_csv('../../../test_labels.csv')['Survived']

## Read in Models

In [ ]:
models = {}

for mdl in ['LR', 'SVM', 'MLP', 'RF', 'GB', 'KNN']:
    models[mdl] = joblib.load(f'../../../{mdl}_model.pkl')

In [ ]:
models

## Evaluate models on the validation set

In [ ]:
def evaluate_model(name, model, features, labels):
    start = time()
    pred = model.predict(features)
    end = time()
    accuracy = round(accuracy_score(labels, pred), 3)
    precision = round(precision_score(labels, pred), 3)
    recall = round(recall_score(labels, pred), 3)
    print('{} -- Accuracy: {} / Precision: {} / Recall: {} / Latency: {}ms'.format(name,
                                                                                   accuracy,
                                                                                   precision,
                                                                                   recall,
                                                                                   round((end - start)*1000, 1)))

## Evaluate best model on test set

In [ ]:
for name, mdl in models.items():
    evaluate_model(name, mdl, val_features, val_labels.array)

Output:
  
`LR -- Accuracy: 0.77 / Precision: 0.707 / Recall: 0.631 / Latency: 13.0ms  
 SVM -- Accuracy: 0.747 / Precision: 0.672 / Recall: 0.6 / Latency: 4.0ms  
 MLP -- Accuracy: 0.77 / Precision: 0.707 / Recall: 0.631 / Latency: 60.0ms  
 RF -- Accuracy: 0.781 / Precision: 0.724 / Recall: 0.646 / Latency: 10.0ms  
 GB -- Accuracy: 0.815 / Precision: 0.808 / Recall: 0.646 / Latency: 4.0ms  
 KNN -- Accuracy: 0.708 / Precision: 0.603 / Recall: 0.585 / Latency: 7.0ms`

So for me Gradient Boosted is the best. How does it do on the held-back test data?

In [ ]:
evaluate_model('Gradient Boosted', models['GB'], te_features, te_labels.array)

Output:
  
  `Gradient Boosted -- Accuracy: 0.816 / Precision: 0.852 / Recall: 0.684 / Latency: 3.0ms`

So the performance on the held-back test set is very similar to the performance on the validation set, in fact it is a little better. 